In [ ]:
from __future__ import print_function

import os
import time
import numpy as np
import pandas as pd

from string import punctuation
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS
from Utils.util_Functions import *
from Utils.Read_Corpus import *

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report, \
accuracy_score, precision_score, recall_score, fbeta_score, make_scorer, log_loss, classification_report

import sys
 
from text_preprocessing.text_preprocessing import RemoveWordsTransform, CleanTransform, LemmatizeTransform, StandardizeTransform


import matplotlib.pyplot as plt
from scipy import stats
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
from scipy import stats
from sklearn import linear_model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import BaggingClassifier
from scipy.sparse import csc_matrix, vstack
from scipy.stats import entropy
from collections import Counter
import copy
from multiprocessing import Pool
from sklearn.metrics import average_precision_score
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csc_matrix, vstack
from scipy.stats import entropy
from collections import Counter
from active_learning import ActiveLearner
from sklearn.neighbors import kneighbors_graph
from sklearn import metrics

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler,MaxAbsScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.linear_model import RidgeCV
from sklearn import neighbors
from IPython.display import clear_output



import logging
import math

print(__doc__)

from scipy.spatial import distance
import scipy 
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import cross_validate


#from sklearn.metrics import pairwise_distance
from sklearn.metrics.pairwise import pairwise_distances
import warnings
warnings.filterwarnings('ignore')

In [ ]:
stopwords_nltk = stopwords.words('english')
punctuation = list(punctuation)
stopwords_gensim = []
for i in STOPWORDS:
    stopwords_gensim.append(i)
nltk_gensim_stop_words = np.union1d(stopwords_nltk,stopwords_gensim)
nltk_gensim_stop_words = nltk_gensim_stop_words.tolist()

processing = Pipeline([
    ('clean', CleanTransform(parallel=False)),
    ('lemmatize', LemmatizeTransform(parallel=False)),
    ('stop_word', RemoveWordsTransform(parallel=False,non_info_words=nltk_gensim_stop_words))
])



data, train, y_train = read_dataset(4)
abstract = np.array(data['Abstract']).astype(str)
title = np.array(data['Title']).astype(str)
label_encoder = LabelEncoder().fit(y_train)
y_train_bin = label_encoder.transform(y_train)


index, = np.where(y_train)
nb_corpus=20
p_train = nb_corpus / len(index)
rd_state=12345

p_train = nb_corpus / len(index)

n_total_samples = nb_corpus
max_iterations = 20
n_priotirized_samples = 5

index_train1, index_test1 = train_test_split(index, test_size=1 - p_train, stratify=y_train_bin[index],
                                               random_state=rd_state)
    
%time X_train = processing.transform(train)

%time titles = processing.transform(title)
%time abstracts = processing.transform(abstract)

X_train = np.array(X_train)
titles = np.array(titles)
abstracts = np.array(abstracts)


X_train_final,title_train_final,abstract_train_final, y_train_final = X_train[index_train1],titles[index_train1],abstracts[index_train1], y_train_bin[index_train1]
X_test_final,title_test_final,abstract_test_final, y_test_final = X_train[index_test1],titles[index_test1],abstracts[index_test1], y_train_bin[index_test1]

print("All data :", abstract.shape, y_train.shape)
print("Train Data :", abstract_train_final.shape, y_train_final.shape)
print("Test Data : ", abstract_test_final.shape, y_test_final.shape)

indexes = [index, index_train1, index_test1]
label = ['all', 'train', 'test']
# check if the probability of each class is respected in a train/test set like the starting dataset.
p_included = 0
for j in range(len(indexes)):
        tt = y_train_bin[indexes[j]]
        ct = pd.DataFrame()
        ct['Count'] = np.bincount(tt)
        ct['Freq'] = ct['Count'] / sum(np.bincount(tt)) * 100
        if (j == 0):
            freq = ct['Freq'].values
            p_included = freq[1]
        print('__________________________________________')
        print("\t*", label[j])
        print()
        print(ct)
        print()

In [ ]:
df_iter = 0

from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
processing = Pipeline([
    ('clean', CleanTransform(parallel=False)),
     ('lemmatize', LemmatizeTransform(parallel=False)),
     ('stop_word', RemoveWordsTransform(parallel=False,non_info_words=nltk_gensim_stop_words))
])

nb_corpus = 20
rd_state = 12345
score = {'kappa': make_scorer(cohen_kappa_score)}
#score = make_scorer(cohen_kappa_score)
#scores = {'recall': 'precision_micro'}
gs_models = []
pred_train = []
pred_test = []

nb_corpus=40
p_train = nb_corpus / len(index)
rd_state=12345

p_train = nb_corpus / len(index)

n_total_samples = nb_corpus
max_iterations = 25#350#3#50

n_priotirized_samples = 40

index_train = index_train1.copy()
index_test = index_test1.copy()


#X_train = Compute_tf_idf(X_train)

X_train_final,title_train_final,abstract_train_final, y_train_final =X_train[index_train], titles[index_train],abstracts[index_train], y_train_bin[index_train]
X_test_final,title_test_final,abstract_test_final, y_test_final = X_train[index_test],titles[index_test],abstracts[index_test], y_train_bin[index_test]

X_train_lab= np.array(X_train)[index_train.astype(np.int32)]
X_test_lab = np.array(X_train)[index_test.astype(np.int32)]

titles_train_lab= np.array(title)[index_train.astype(np.int32)]
titles_test_lab = np.array(title)[index_test.astype(np.int32)]

abstract_train_lab= np.array(abstract)[index_train.astype(np.int32)]
abstract_test_lab = np.array(abstract)[index_test.astype(np.int32)]




print("All data :", X_train.shape, y_train.shape)
print("Train Data :", X_train_final.shape, y_train_final.shape)
print("Test Data : ", X_test_final.shape, y_test_final.shape)

indexes = [index, index_train, index_test]
label = ['all', 'train', 'test']
            # check if the probability of each class is respected in a train/test set like the starting dataset.
p_included = 0
for j in range(len(indexes)):
            tt = y_train_bin[indexes[j]]
            ct = pd.DataFrame()
            ct['Count'] = np.bincount(tt)
            ct['Freq'] = ct['Count'] / sum(np.bincount(tt)) * 100
            if (j == 0):
                freq = ct['Freq'].values
                p_included = freq[1]
            print('__________________________________________')
            print("\t*", label[j])
            print()
            print(ct)
            print()

pred_train = []
sample_indice =  []
sample_index =  []              
pred_test = []
already_selected = []
min_distances = None

#############################################################

pipelineSVM = Pipeline([('tfidf', TfidfVectorizer(min_df=3, use_idf=True,ngram_range=(1,3))),
                        ('normalizer', Normalizer(copy=False)),
                        # ('standard_scaler', StandardScaler(with_mean=False)),
                        ('SVM', LinearSVC(class_weight='balanced'))])


#############################################################

pipelineLogR = Pipeline([('tfidf', TfidfVectorizer(min_df=3, use_idf=True,ngram_range=(1,3))),
                         ('normalizer', Normalizer(copy=False)),
                         # ('standard_scaler', StandardScaler(with_mean=False)),
                         ('logR', LogisticRegression(class_weight='balanced'))])

#############################################################

pipelineSGD = Pipeline([('tfidf', TfidfVectorizer(min_df=2, use_idf=True,ngram_range=(1,3))),
                        ('normalizer', Normalizer(copy=False)),
                        ('standard_scaler', StandardScaler(with_mean=False)),
                         #('standard_scaler', MaxAbsScaler()),
                        ('SgdSVM', SGDClassifier(class_weight='balanced'))])

#############################################################


pipelineRC = Pipeline([('tfidf', TfidfVectorizer(min_df=4, use_idf=True,ngram_range=(1,3))),
                       # ('standard_scaler', StandardScaler(with_mean=False)),
                        ('normalizer', Normalizer(copy=False)),
                        ('RC', RidgeClassifier(tol=1e-2, class_weight='balanced'))])


#############################################################
pipelines = [pipelineLogR,pipelineSVM,pipelineSGD,pipelineRC] 

#############################################################

parametersSVM = {
    'tfidf__norm': [None, 'l2'],
    #'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_df': [0.6],
    'tfidf__max_features' : [500],
    'tfidf__stop_words': [nltk_gensim_stop_words],
    'SVM__C': [0.01, 0.001,0.0001,0.00001]
}
    
#############################################################

parametersLogR = {
    'tfidf__norm': [None, 'l2'],
    #'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_df': [0.6],
    'tfidf__max_features' : [600],
    'tfidf__stop_words': [nltk_gensim_stop_words],
    'logR__C': [0.01, 0.001,0.0001,0.00001]
}

#############################################################

parametersSGD = {
    'tfidf__norm': [None, 'l2'],
    #'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_df': [0.4],
    'tfidf__max_features' : [500],
    'tfidf__stop_words': [nltk_gensim_stop_words],
    'SgdSVM__alpha': [0.1,0.01, 0.001],
    'SgdSVM__loss': ['hinge', 'log']
}

#############################################################

parametersRC = {
    'tfidf__norm': [ None,'l2'],
    #'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_df': [0.5],
    #'tfidf__max_features' : [500],
    'tfidf__stop_words': [nltk_gensim_stop_words],
    'RC__alpha': [0.1,1,10,100,1000]
}

#############################################################
parameters = [parametersLogR,parametersSVM,parametersSGD,parametersRC]


gs_df = pd.DataFrame(columns=['Iteration','Data', 'Proba Included', 'Scoring', 'Pipeline', 'Train data',
                                      'Train Excluded data','Train Included data',
                                      'Best score',
                                      'Train F1','Train Accuracy', 'Train Recall','Train precision', 'Train FNR', 'Train TPR',
                                      'Test data','Test Excluded data','Test Included data',
                                      'Test F1', 'Test Kappa','Test Accuracy', 'Test Recall','Test precision', 
                                      'Test FNR','Test TPR','Test FPR','Test TNR',
                                      'AUC_ROC','FP','FN',
                                      'Best parameters', 'Model'])


  
Metrics = ['recall','recall',make_scorer(cohen_kappa_score),make_scorer(cohen_kappa_score)]


init=0

Unlabeled_pool = X_test_final
Unlabeled_views = [X_test_final,abstract_test_final,title_test_final,title_test_final]

Labeled_pool = X_train_final
Labeled_views = [X_train_final,abstract_train_final,title_train_final,title_train_final]

#############################################################
CoTesting_args = dict(disagreement='vote',selectQuery='random')
# disagreement : [vote, kl_divergence]
# selectQuery : [random, exploration, conservative, aggressive]
#############################################################
QuerybyCommittee_args = dict(disagreement='vote')
# disagreement : [vote, kl_divergence]
#############################################################
ExpectedModelChange_args =dict()
#############################################################
WeightedExpectedModelChange_args =dict()
#############################################################
ExpectedErrorReduction_args= dict(loss='log')
# loss : [log, 01]
#############################################################
QuerybyDiversity_args= dict(dist='cosine',beta=0.15)
# dist : [cosine, euclidean, seuclidean, sqeuclidean, hamming, cityblock, braycurtis, canberra ]
# beta : ]0,1[
#############################################################
DiversityClustering_args= dict(dist='cosine',beta=0.1)
# dist : [cosine, euclidean, seuclidean, sqeuclidean, hamming, cityblock, braycurtis, canberra ]
# beta : ]0,1[
#############################################################
UncertaintyClustering_args= dict()
#############################################################
UncertaintySampling_args= dict(heuristique='entropy')
# heuristique : [least_confident, max_margin, entropy]
#############################################################
QuerybyBagging_args = dict(n_bags=5,method='entropy')
#############################################################
QuerybyRandom_args = dict()
#############################################################

from AmarisActiveLearning.AmarisActiveLearning import AL

al = AL('CoTesting',CoTesting_args,
        X_train,
        y_train_bin,
        Unlabeled_pool,Unlabeled_views,
        Labeled_pool,Labeled_views,
        index_test,index_train,
        y_train_final,y_test_final,
        0,
        pipelines,
        10,
        50,
        500,
        parameters,
        Metrics,
        GridSearchCV)


prep = False
pips = ['ActiveLearning']
pips_name = ['ActiveLearning']
scores = ['recall']
df_iter=0
f1_test_ = np.array([])
kappa_test_ = np.array([])
precision_test_ = np.array([])
FNR_test_ = np.array([])
TPR_test_ = np.array([])
FPR_test_ = np.array([])
TNR_test_ = np.array([])
roc_auc_ = np.array([])
accuracy_score_test_ = np.array([])
recall_test_ = np.array([])
for it in range(al.n_iter):
        pred_test = []
        index_train = al.index_labeled_pool
        index_test = al.index_unlabeled_pool
        indexes = [index, al.index_labeled_pool, al.index_unlabeled_pool]
        label = ['all', 'train', 'test']
        tt = y_train_bin[indexes[1]]

        ct = pd.DataFrame()
        ct['Count'] = np.bincount(tt)
        excluded_train = ct['Count'][0]
        included_train = ct['Count'][1]
        tt = y_train_bin[indexes[2]]
        ct = pd.DataFrame()
        ct['Count'] = np.bincount(tt)
        excluded_test = ct['Count'][0]
        included_test = ct['Count'][1]
        
        
        time_traitement_dataset = time.time()
        
        n_cv =10                    
        
        al.train(n_cv)
                        
         #--------------------
                        
        y_test_pred = al.predict(al.Unlabeled_pool)    
        y_test_true = al.y_unlabeled
        cm = confusion_matrix(y_test_true, y_test_pred)

        tn = cm[0][0]
        fn = cm[1][0]
        fp = cm[0][1]
        tp = cm[1][1]
        roc_auc = roc_auc_score(y_test_true, y_test_pred)
                        
        A = (tp*tn) - (fp*fn)
        B = np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
        MCC = A/B


        TPR_test = tp/(tp+fn)
        TNR_test = tn/(tn+fp)
        FNR_test = fn/(tp+fn)
        FPR_test = fp/(fp+tn)

        precision_test = tp/(tp+fp)
        recall_test = tp/(tp+fn)
        FNR_test = 1 - recall_test
        FPR_test = 1 - TNR_test
        accuracy_score_test = (tp+tn)/(tp+fp+tn+fn)
        f1_test = 2 *(precision_test*recall_test)/(precision_test+recall_test)
        kappa_test = cohen_kappa_score(y_test_true, y_test_pred, labels=None, weights=None)

                        
        pred_test.append((y_test_final, y_test_pred))

        f1_test_ = np.append(f1_test_,f1_test)
        kappa_test_ = np.append(kappa_test_,kappa_test)

        precision_test_ = np.append(precision_test_,precision_test)
        TNR_test_ = np.append(TNR_test_,TNR_test)
        FNR_test_ = np.append(FNR_test_,FNR_test)
        TPR_test_ = np.append(TPR_test_,TPR_test)
        FPR_test_ = np.append(FPR_test_,FPR_test)
        roc_auc_ = np.append(roc_auc_,roc_auc)

        accuracy_score_test_ = np.append(accuracy_score_test_,accuracy_score_test)
        recall_test_ = np.append(recall_test_,recall_test)
        
        ###


        df_iter += 1
        gs_df.loc[df_iter] = [it,'RS' + str(df_iter), p_included, score, 'pips_name[p]',(al.Labeled_pool.shape[0])
                              ,len(np.where(al.y_labeled==0)),len(np.where(al.y_labeled==1)),'',
                              'np.mean(f1_train_)', 'np.mean(accuracy_score_train_)', 'np.mean(recall_train_)',
                              'np.mean(precision_train_)', 'TNR_train', 'TPR_train',
                              y_test_final.shape[0],
                              len(np.where(al.y_unlabeled==0)),len(np.where(al.y_unlabeled==1)),np.mean(f1_test_),np.mean(kappa_test_),
                              np.mean(accuracy_score_test_),np.mean(recall_test_),np.mean( precision_test_), 
                              np.mean(FNR_test_),np.mean(TPR_test_),
                    np.mean(FPR_test_),np.mean(TNR_test_),np.mean(roc_auc_),fp,fn,al.classifiers[0], MCC]

                  
        if (it< al.n_iter-1) :
            
                # change the parameters of the strategy without changing the strategy itself.
  
                strategy = None
                args = {'disagreement' : 'vote'}
                if(len(al.index_labeled_pool)< 140):
                    args['selectQuery'] = 'exploration'
                else :
                    args['selectQuery'] = 'random'

                selected_index = al.make_query(strategy,args)
                
    
                al.update(selected_index)
                gs_df.to_csv('Active_Learning_ds4'+str(len(al.y_labeled))+'_.csv')
                
        else :
                
                df = pd.read_csv('Active_Learning_ds4_.csv')
                
                plt.figure(figsize=(20,10))
                
                plt.subplot(121)
                plt.plot(df['Train data'],df['Test TPR'], linestyle='-',color='orange',label='AL')

                plt.title('True Positive Rate')
                plt.subplot(122)
                plt.plot(df['Train data'],df['Test TNR'], linestyle='-',color='orange',label='AL')
                plt.title('True Negative Rate')
                plt.show()
